In [ ]:
# MOOn STAndard RaDio flux calculator (MOOSTARD)
# Authors: Kaustubh Hakim, Apurva V Oza
# Citation: Narang et al. (2022) The Astrophysical Journal

import numpy as np
from astropy import units as u
u.imperial.enable()
from astropy.constants import mu0, N_A, R_jup, M_jup, GM_jup, G
m_io   = 8.932e25 * u.g  # m_io: mass of Io (g)
r_io   = 1821e5 * u.cm  # r_io: radius of Io (cm)
# amu = 1 / N_A * u.g / u.mol  # 1 / Avogadro's no.
import matplotlib.pyplot as plt

In [ ]:
# Escape speed
def escape_speed (m_s = 1 * u.Unit(1),         # m_s: ratio of mass of satellite to mass of Io
                  r_s = 1 * u.Unit(1)          # r_s: ratio of radius of satellite to radius of Io
                 ): 

    v_ej  = np.sqrt(2 * G * m_s * m_io / (r_s * r_io)).decompose().cgs

    return v_ej

In [ ]:
# Orbital speed
def orbital_speed (m_p = 1 * u.Unit(1),  # m_p: ratio of planet mass to jupiter mass
                   a_s = 6 * u.Unit(1),  # a_s: ratio of semimajor axis of satellite to planet radius
                   r_p = 1 * u.Unit(1)   # r_p: ratio of planet radius to jupiter radius
                  ):

    d_orb  = (a_s * r_p * R_jup).cgs
    v_orb  = np.sqrt(G * m_p * M_jup / d_orb).decompose().cgs

    return v_orb

In [ ]:
# Relative plasma speed
def rel_plasma_speed (v_orb = 17e5 * u.cm / u.s,  # v_orb: satellite orbital speed (cm/s)
                      v_ej = 2e5 * u.cm / u.s     # v_ej: ejection speed (cm/s)
                     ):

    v0 = np.abs(v_orb - v_ej)
    
    return v0

In [ ]:
# Volume of plasma torus
def volume_torus (a_s = 6 * u.Unit(1),         # a_s: ratio of semimajor axis of satellite to planet radius
                  r_p = 1 * u.Unit(1),         # r_p: ratio of planet radius to jupiter radius 
                  v_ej = 2e5 * u.cm / u.s,     # v_ej: ejection speed (cm/s)
                  v_orb  = 17e5 * u.cm / u.s   # v_orb: satellite orbital speed (cm/s)
                 ): 

    d_orb  = (a_s * r_p * R_jup).cgs   
    vol_t  = (2 * np.pi**2 * d_orb**3 * (v_ej / v_orb)**2).decompose().cgs

    return vol_t

In [ ]:
# Plasma density
def plasma_density (Mdot_s = 2e7 * u.g / u.s,    # Mdot_s: mass loss rate (g/s)
                    t_ion = 3e4 * u.s,           # t_ion: ionisation timescale (s)
                    vol_t = 2e31 * (u.cm)**3     # vol_t: volume of torus (cm3)
                   ):

    rho_s  = (Mdot_s * t_ion / vol_t).decompose().cgs
    
    return rho_s

In [ ]:
# Planetary magnetic field from cyclotron frequency
def magnetic_field_p (nu_c = 50 * u.MHz    # nu_c: cyclotron frequency (MHz)
                     ):
    
    B_p    = nu_c / 2.8 * u.G / u.MHz
    
    return B_p

In [ ]:
# Planetary magnetic field at the location of satellite
def magnetic_field_s (B_p = 50 * u.Gauss,      # B_p: planetary magnetic field (G)
                      a_s = 2 * u.Unit(1)      # a_s: ratio of semimajor axis of satellite to planet radius
                     ):
    
    B_s    = B_p / a_s**3
    
    return B_s

In [ ]:
# Radio power
def power_rad (beta_s = 0.1 * u.Unit(1),           # beta_s: efficiency coefficient
               r_s = 1 * u.Unit(1),                # r_s: ratio of radius of satellite to radius of Io
               B_s = 1 * u.Gauss,                  # B_s: planetary magnetic field at satellite location (G)
               v0 = 10 * u.cm / u.s,               # v0: plasma speed relative to satellite
               rho_s = 1.96e-14 * u.g / (u.cm)**3  # rho_s: satellite plasma density (g/cm3), Io = 7e-20 g/cm3 
              ):
    
    P_rad  = (np.pi * beta_s * (r_s*r_io)**2 * B_s**2 * v0 / mu0 * 
              np.sqrt(rho_s / (rho_s + (1/mu0 * (B_s / v0)**2)))).decompose().cgs
    
    return P_rad

In [ ]:
# Radio Flux, equation 9, Narang et al. (2022)
def flux_rad (P_rad = 100 * u.erg / u.s,    # P_rad: Satellite radio power (erg/s)
                omega = 0.16 * u.Unit(1),   # omega: soild angle through which source power is emitted
                d = 19.3 * u.pc.cgs,        # d: distance to the object (parsec)
                nu_c = 140 * u.MHz          # nu_c: cyclotron frequency (MHz)
               ):

    S_nu = (P_rad / (omega * (nu_c / 2) * d**2)).to('mJy')
    
    return S_nu

In [ ]:
# Fixed parameters for WASP 49 b, HAT-P 12 b, HD 186733 b (Do not modify)

# WASP 49 b (exoplanet archive - Stassun et al. 2017)
p_WASP49  = 2.782 * u.d    # orbital period (days)
d_WASP49  = 194.55 * u.pc  # distance to the object (parsec) 
m_WASP49  = 0.399          # ratio of planet mass to jupiter mass 
r_WASP49  = 1.198          # ratio of planet radius to jupiter radius

# HAT-P-12b (exoplanet archive - Öztürk & Erdem 2019)
p_HATP12  = 3.213 * u.d    # orbital period (days)
d_HATP12  = 142.75 * u.pc  # distance to the object (parsec) 
m_HATP12  = 0.211          # ratio of planet mass to jupiter mass 
r_HATP12  = 0.949          # ratio of planet radius to jupiter radius

# HD 189733 b (exoplanet archive - Addison et al. 2019)
p_HD189  = 2.219 * u.d     # orbital period (days)
d_HD189  = 19.76 * u.pc    # distance to the object (parsec) 
m_HD189  = 1.166           # ratio of planet mass to jupiter mass 
r_HD189  = 1.119           # ratio of planet radius to jupiter radius

In [ ]:
# Plasma parameters from DISHOOM 

# WASP 49 b 
t_WASP49  = 32586 * u.s         # ionization timescale (s)
Md_WASP49 = 10**7.3 * u.g / u.s # mass loss rate (g/s)

# HAT-P-12b 
t_HATP12  = 33162 * u.s         # ionization timescale (s)
Md_HATP12 = 10**8.8 * u.g / u.s # mass loss rate (kg/s)

# HD 189733 b
t_HD189  = 49867 * u.s          # ionization timescale (s)
Md_HD189 = 10**8.4 * u.g / u.s  # mass loss rate (kg/s)

In [ ]:
# Generic parameters

# Following parameters based on observational frequency
nu_c = 150 * u.MHz            # cyclotron frequency (MHz)
B_p  = magnetic_field_p(nu_c) # planetary magnetic field (Gauss)

# Following parameters can be tweaked
r_s    = 1  * u.Unit(1)       # ratio of satellite radius to Io radius (Earth radius = 3.5 * Io radius)
beta_s = 0.1 * u.Unit(1)     # beta_s: efficiency coefficient

In [ ]:
# WASP 49 -- Radio Flux

# Stable satellite orbits
a_WASP49  = 1.19 * u.Unit(1) # a_Hill=1.74 # a_Roche=1.19 # ratio of semimajor axis of satellite to planet radius

# Magnetic field at satellite location
Bs_WASP49 = magnetic_field_s(B_p = B_p, a_s = a_WASP49) # planetary magnetic field at satellite location (Gauss)

# Speeds
vej_WASP49  = escape_speed() # equal to io
vorb_WASP49 = orbital_speed(m_p = m_WASP49, a_s = a_WASP49, r_p = r_WASP49)
v0_WASP49   = rel_plasma_speed(v_ej = vej_WASP49, v_orb = vorb_WASP49)

# Torus volume and plasma density
vol_WASP49 = volume_torus (a_s = a_WASP49, r_p = r_WASP49, v_ej = vej_WASP49, v_orb = vorb_WASP49)
rho_WASP49 = plasma_density(Mdot_s = Md_WASP49, t_ion = t_WASP49, vol_t = vol_WASP49)

# Radio power
Prad_WASP49 = power_rad(beta_s = beta_s, r_s = r_s, B_s = Bs_WASP49, v0 = v0_WASP49, rho_s = rho_WASP49)

# Radio flux
S_WASP49  = flux_rad(P_rad = Prad_WASP49, d = d_WASP49, nu_c = nu_c)

print('WASP 49 b:  S =', S_WASP49.round(3),'for a_s =', a_WASP49.round(2), 
      'R_P and r_s =', r_s, 'R_io and beta_s =', beta_s.round(2), 'at f =', nu_c)

In [ ]:
# HAT-P-12b -- Radio Flux

# Stable satellite orbits
a_HATP12  = 1.14 * u.Unit(1) # a_Hill=1.86 # a_Roche=1.14 # ratio of semimajor axis of satellite to planet radius

# Magnetic field at satellite location
Bs_HATP12 = magnetic_field_s(B_p = B_p, a_s = a_HATP12) # planetary magnetic field at satellite location (Gauss)

# Speeds
vej_HATP12  = escape_speed() # equal to io
vorb_HATP12 = orbital_speed(m_p = m_HATP12, a_s = a_HATP12, r_p = r_HATP12)
v0_HATP12   = rel_plasma_speed(v_ej = vej_HATP12, v_orb = vorb_HATP12)

# Torus volume and plasma density
vol_HATP12 = volume_torus (a_s = a_HATP12, r_p = r_HATP12, v_ej = vej_HATP12, v_orb = vorb_HATP12)
rho_HATP12 = plasma_density(Mdot_s = Md_HATP12, t_ion = t_HATP12, vol_t = vol_HATP12)

# Radio power
Prad_HATP12 = power_rad(beta_s = beta_s, r_s = r_s, B_s = Bs_HATP12, v0 = v0_HATP12, rho_s = rho_HATP12)

# Radio flux
S_HATP12  = flux_rad(P_rad = Prad_HATP12, d = d_HATP12, nu_c = nu_c)

print('HAT-P 12 b:  S =', S_HATP12.round(3),'for a_s =', a_HATP12.round(2), 
      'R_P and r_s =', r_s, 'R_io and beta_s =', beta_s.round(2), 'at f =', nu_c)

In [ ]:
# HD 189733 b -- Radio Flux

# Stable satellite orbits
a_HD189  = 1.14 * u.Unit(1) # a_Hill=2.11 # a_Roche=1.14 # ratio of semimajor axis of satellite to planet radius

# Magnetic field at satellite location
Bs_HD189 = magnetic_field_s(B_p = B_p, a_s = a_HD189) # planetary magnetic field at satellite location (Gauss)

# Speeds
vej_HD189  = escape_speed() # equal to io
vorb_HD189 = orbital_speed(m_p = m_HD189, a_s = a_HD189, r_p = r_HD189)
v0_HD189   = rel_plasma_speed(v_ej = vej_HD189, v_orb = vorb_HD189)

# Torus volume and plasma density
vol_HD189 = volume_torus (a_s = a_HD189, r_p = r_HD189, v_ej = vej_HD189, v_orb = vorb_HD189)
rho_HD189 = plasma_density(Mdot_s = Md_HD189, t_ion = t_HD189, vol_t = vol_HD189)

# Radio power
Prad_HD189 = power_rad(beta_s = beta_s, r_s = r_s, B_s = Bs_HD189, v0 = v0_HD189, rho_s = rho_HD189)

# Radio flux
S_HD189  = flux_rad(P_rad = Prad_HD189, d = d_HD189, nu_c = nu_c)

print('HD 189733 b:  S =', S_HD189.round(2),'for a_s =', a_HD189.round(2), 
      'R_P and r_s =', r_s, 'R_io and beta_s =', beta_s.round(2), 'at f =', nu_c)